In [28]:
import random
import string
from pathlib import Path
import os
import requests
import shutil

from git import Repo
from bs4 import BeautifulSoup as Soup
import openai
from PIL import Image

In [29]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [30]:
PATH_TO_BLOG_REPO = Path("/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/.git")

In [31]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [32]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [33]:
PATH_TO_CONTENT

PosixPath('/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/content')

In [34]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [35]:
def random_string(length):
    letters = string.ascii_lowercase + "\n"
    
    return ''.join(random.choice(letters) for i in range(length))

random_string(10)

'sclxvnqbss'

In [36]:
def update_blog(commit_message="Updated blog"):
    repo = Repo(PATH_TO_BLOG_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

In [37]:
random_blogpost = random_string(1000)

In [38]:
with open(PATH_TO_BLOG/"index.html", "w") as f:
    f.write(random_blogpost)

In [39]:
update_blog()

In [50]:
def create_new_blog(title,content,cover_image):
    cover_image= Path(cover_image)
    
    # auto file naming utility
    files = len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image,PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        # WRITE A NEW HTML FILE
        with open(path_to_new_content,"w") as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>")
            # OpenAI --->> Completion GPT -->> "hello\nblog post\n"
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog created")
            return path_to_new_content
    else:
        raise FileExistsError("File already exists, please check again your name! Aborting!")

In [51]:
path_to_new_content = create_new_blog('Test_title','thisisatest','templogo.png')

Blog created


In [17]:
# index.html --> Blog posts

In [52]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [53]:
str(soup)

'<!DOCTYPE html>\n\n<html>\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<title>Document</title>\n<meta content="" name="description"/>\n<meta content="width=device-width, initial-scale=1" name="viewport"/>\n<link href="" rel="stylesheet"/>\n</head>\n<body>\n<h1>My Blog Home Page</h1>\n<a href="index.html">Home</a>\n</body>\n</html>'

In [21]:
# checking duplicate links

In [54]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    # /pathtocontent/1.html check for overwriting
    return content_path in urls

In [55]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/"index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))

In [56]:
write_to_index(path_to_new_content)

In [57]:
update_blog()

In [58]:
def create_prompt(title):
    prompt = """
    Biography: My name is Kristin and I am a Software Engineer with a emphasis on Devops practices.
    
    Blog
    Title: {}
    tags: tech, devops, AI, python, observability, coding
    Summary: I talk about what the future of AI could hold for Devops
    Full Text: """.format(title)
    return prompt

In [59]:
title = "The future of devops and AI"
print(create_prompt(title))


    Biography: My name is Kristin and I am a Software Engineer with a emphasis on Devops practices.
    
    Blog
    Title: The future of devops and AI
    tags: tech, devops, AI, python, observability, coding
    Summary: I talk about what the future of AI could hold for Devops
    Full Text: 


In [60]:
response = openai.Completion.create(engine='text-davinci-003',
                                    prompt=create_prompt(title),
                                    max_tokens=1000,
                                    temperature=0.7)

In [61]:
blog_content = response['choices'][0]['text']

In [62]:
print(blog_content)


    With the advent of AI technology, Devops engineers have been faced with the challenge of efficiently managing software workloads with limited resources. This has spawned a new breed of software engineering, DevOps AI, which is focused on optimizing the performance of applications by using machine learning algorithms to automate the processes associated with software deployment and operations. 

The combination of Devops and AI technologies has the potential to revolutionize the way software is deployed and operated. AI can be used to detect anomalies in the system, predict errors and automate processes, allowing developers to focus on the core tasks of building and deploying software. AI can also be used to optimize the performance of applications, making them more efficient and reliable. 

AI can also help reduce the cost of Devops by automating manual processes such as system monitoring and code deployment. This can lead to more efficient use of resources and faster delivery of 

In [69]:
def dalle2_prompt(title):
    prompt = f"a watercolor painting showing {title}"
    return prompt

In [70]:
image_prompt =  dalle2_prompt(title)

In [71]:
image_prompt

'a watercolor painting showing The future of devops and AI'

In [72]:
response = openai.Image.create(prompt=image_prompt,
                              n=1,size="512x512")

In [73]:
image_url = response['data'][0]['url']

In [74]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-ZJxtr8k7uUYQka7iCmvXgZay/user-P0RAWNfvL0Wnea1OjOcqsSUc/img-hipwTZsXAanTzNcCXRw50jnB.png?st=2023-03-21T21%3A10%3A44Z&se=2023-03-21T23%3A10%3A44Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-03-21T16%3A01%3A47Z&ske=2023-03-22T16%3A01%3A47Z&sks=b&skv=2021-08-06&sig=H7TAjCMg6vzI8kemI1P2j/WGm5pTl41NdHDqHclBeg4%3D'

In [75]:
def save_image(image_url, file_name):
    image_res = requests.get(image_url, stream = True)
    
    if image_res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print("Error downloading image!")
    return image_res.status_code

In [76]:
save_image(image_url,file_name="title2.png")

200

In [77]:
title

'The future of devops and AI'

In [78]:
blog_content

"\n    With the advent of AI technology, Devops engineers have been faced with the challenge of efficiently managing software workloads with limited resources. This has spawned a new breed of software engineering, DevOps AI, which is focused on optimizing the performance of applications by using machine learning algorithms to automate the processes associated with software deployment and operations. \n\nThe combination of Devops and AI technologies has the potential to revolutionize the way software is deployed and operated. AI can be used to detect anomalies in the system, predict errors and automate processes, allowing developers to focus on the core tasks of building and deploying software. AI can also be used to optimize the performance of applications, making them more efficient and reliable. \n\nAI can also help reduce the cost of Devops by automating manual processes such as system monitoring and code deployment. This can lead to more efficient use of resources and faster delive

In [79]:
path_to_new_content = create_new_blog(title,blog_content,'title2.png')

Blog created


In [80]:
write_to_index(path_to_new_content)

In [81]:
update_blog()